In [1]:
x=3

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
from bs4 import BeautifulSoup

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def clean_data(df):
    # Sort by column: 'DoD Chg Tight Bid >3mm' (descending)
    df = df.sort_values(['DoD Chg Tight Bid >3mm'], ascending=[False])
    return df

# Load data (first sheet only)
df = pd.read_excel(r'c:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\Bond-RV-App\analytics\processed_data\portfolio_runs_view.xlsx', sheet_name=0)

df_clean = clean_data(df.copy())
df_original = df_clean.copy()  # Keep original for reset

def create_sortable_table(df):
    """Create interactive table with sort buttons for each column"""
    
    # Inject comprehensive dark mode CSS for widgets and DataFrame
    dark_mode_css = """
    <style>
        /* Global dark mode for all widgets */
        .widget-label {
            color: #d4d4d4 !important;
        }
        .widget-button {
            background-color: #3c3c3c !important;
            color: #d4d4d4 !important;
            border: 1px solid #555555 !important;
        }
        .widget-button:hover {
            background-color: #4c4c4c !important;
        }
        .widget-button:active {
            background-color: #2c2c2c !important;
        }
        .widget-button.btn-warning {
            background-color: #856404 !important;
            color: #fff !important;
        }
        .widget-button.btn-warning:hover {
            background-color: #996f0a !important;
        }
        /* Single table container */
        .single-table-container {
            width: 100% !important;
            overflow-x: auto !important;
            background-color: #1e1e1e !important;
        }
        .single-table {
            background-color: #1e1e1e !important;
            color: #d4d4d4 !important;
            border-collapse: separate !important;
            border-spacing: 0 !important;
            width: 100% !important;
            table-layout: fixed !important;
        }
        .single-table thead th {
            background-color: #2d2d2d !important;
            color: #ffffff !important;
            border: 1px solid #555555 !important;
            white-space: nowrap !important;
            padding: 8px !important;
            position: sticky !important;
            top: 0 !important;
            z-index: 5 !important;
        }
        .single-table tbody td {
            background-color: #1e1e1e !important;
            color: #d4d4d4 !important;
            border: 1px solid #555555 !important;
            white-space: nowrap !important;
            padding: 8px !important;
        }
        .single-table tbody tr:nth-child(even) {
            background-color: #252525 !important;
        }
        .single-table tbody tr:hover {
            background-color: #3c3c3c !important;
        }
        .single-table .sticky-security {
            position: sticky !important;
            left: 0 !important;
            background-color: #1e1e1e !important;
            border-right: 2px solid #666666 !important;
            z-index: 6 !important;
        }
        .single-table thead .sticky-security {
            z-index: 7 !important;
            box-shadow: 2px 0 0 #666666 !important;
        }
        /* Dark mode for output area */
        .output_area {
            background-color: #1e1e1e !important;
        }
        /* Dark mode for HBox/VBox containers */
        .widget-hbox, .widget-vbox {
            background-color: #1e1e1e !important;
        }
        /* Dark mode for range sliders */
        .widget-slider .ui-slider-handle {
            background-color: #6495ed !important;
            border: 1px solid #555555 !important;
        }
        .widget-slider .ui-slider-track {
            background-color: #3c3c3c !important;
            border: 1px solid #555555 !important;
        }
        .widget-slider .ui-slider-range {
            background-color: #6495ed !important;
        }
        .widget-label {
            color: #d4d4d4 !important;
        }
        /* White text for slider value displays */
        .widget-slider .widget-readout,
        .widget-slider .widget-readout *,
        .widget-slider .widget-readout input {
            color: #ffffff !important;
        }
        .widget-slider .widget-readout input {
            background-color: #1e1e1e !important;
            border: 1px solid #555555 !important;
        }
        .widget-slider label,
        .widget-slider .widget-label,
        .widget-slider .widget-inline-hbox,
        .widget-slider .widget-inline-hbox * {
            color: #ffffff !important;
        }
        /* Ensure slider value text is white */
        .widget-intrangeslider .widget-readout,
        .widget-floatrangeslider .widget-readout,
        .widget-intrangeslider .widget-readout *,
        .widget-floatrangeslider .widget-readout * {
            color: #ffffff !important;
        }
        /* Column selector panel styles */
        .column-selector-panel {
            background-color: #1e1e1e !important;
            border: 1px solid #555555 !important;
            border-radius: 4px !important;
            padding: 10px !important;
            max-height: 600px !important;
            overflow-y: auto !important;
            width: 300px !important;
        }
        .column-list-item {
            background-color: #2d2d2d !important;
            border: 1px solid #555555 !important;
            padding: 8px !important;
            margin: 4px 0 !important;
            cursor: move !important;
            display: flex !important;
            align-items: center !important;
            color: #d4d4d4 !important;
            border-radius: 3px !important;
        }
        .column-list-item:hover {
            background-color: #3c3c3c !important;
        }
        .column-list-item.dragging {
            opacity: 0.5 !important;
        }
        .column-list-item.drag-over {
            border-top: 3px solid #6495ed !important;
        }
        .column-checkbox {
            margin-right: 8px !important;
        }
        .column-name {
            flex: 1 !important;
            user-select: none !important;
        }
        .drag-handle {
            margin-right: 8px !important;
            color: #888 !important;
            font-size: 18px !important;
            cursor: move !important;
        }
        .column-selector-header {
            display: flex !important;
            justify-content: space-between !important;
            align-items: center !important;
            margin-bottom: 10px !important;
        }
    </style>
    """
    display(HTML(dark_mode_css))
    
    # Store current sort state
    sort_state = {'column': None, 'ascending': True}
    current_df = df.copy()
    
    # Identify non-numeric columns for filtering
    non_numeric_cols = []
    numeric_cols = []
    for col in df.columns:
        if df[col].dtype == 'object' or col == 'Security':
            non_numeric_cols.append(col)
        else:
            # Check if column is numeric (int or float)
            try:
                pd.to_numeric(df[col], errors='raise')
                numeric_cols.append(col)
            except (ValueError, TypeError):
                pass
    
    # Store filter state: {column: {'mode': 'contains'/'not_contains', 'values': [selected_values]}}
    filter_state = {col: {'mode': 'contains', 'values': []} for col in non_numeric_cols}
    
    # Store range slider state: {column: {'min': min_value, 'max': max_value}}
    range_state = {}
    for col in numeric_cols:
        try:
            numeric_series = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(numeric_series) > 0:
                col_min = float(numeric_series.min())
                col_max = float(numeric_series.max())
                range_state[col] = {'min': col_min, 'max': col_max, 'current_min': col_min, 'current_max': col_max}
        except:
            pass
    
    # Column visibility and order state
    # Default: all columns visible, Security always first and always visible
    default_column_order = list(df.columns)
    # Ensure Security is first if it exists
    if 'Security' in default_column_order:
        default_column_order.remove('Security')
        default_column_order.insert(0, 'Security')
    
    column_order = default_column_order.copy()
    column_visibility = {col: True for col in df.columns}  # All visible by default
    # Security is always visible (enforced in UI)
    
    # Create output widget for the table
    output = widgets.Output()
    
    def format_single_value(value, col_name):
        """Format a single numeric value based on column name rules"""
        if pd.isna(value) or value is None:
            return ""
        
        # Columns with 1 decimal place (case-insensitive matching)
        one_decimal_cols = ['Yrs (Cvn)', 'Yrs (CVN)', 'MTD Equity', 'YTD Equity']
        # Columns with 2 decimal places (no % sign)
        two_decimal_cols = ['Retracement']
        
        # Normalize column name for comparison (case-insensitive)
        col_lower = col_name.lower()
        is_one_decimal = any(one_col.lower() == col_lower for one_col in one_decimal_cols)
        is_two_decimal = any(two_col.lower() == col_lower for two_col in two_decimal_cols)
        
        try:
            num_value = float(value)
            if is_one_decimal:
                return f"{num_value:.1f}"
            elif is_two_decimal:
                return f"{num_value:.2f}"
            else:
                # Round to whole number with thousand separators
                return f"{int(round(num_value)):,}"
        except (ValueError, TypeError):
            return str(value)
    
    def format_numeric_columns(df):
        """Format numeric columns with special cases"""
        df_formatted = df.copy()
        # Exclude text columns like Security
        exclude_cols = ['Security']
        # Columns with 1 decimal place (case-insensitive matching)
        one_decimal_cols = ['Yrs (Cvn)', 'Yrs (CVN)', 'MTD Equity', 'YTD Equity']
        # Columns with 2 decimal places (no % sign)
        two_decimal_cols = ['Retracement']
        
        for col in df_formatted.columns:
            if col in exclude_cols:
                continue
            
            # Normalize column name for comparison (case-insensitive)
            col_lower = col.lower()
            is_one_decimal = any(one_col.lower() == col_lower for one_col in one_decimal_cols)
            is_two_decimal = any(two_col.lower() == col_lower for two_col in two_decimal_cols)
            
            # Handle 1 decimal place columns
            if is_one_decimal:
                if df_formatted[col].dtype in [np.int64, np.int32, np.float64, np.float32]:
                    df_formatted[col] = df_formatted[col].apply(
                        lambda x: f"{x:.1f}" if pd.notna(x) and not pd.isna(x) else ""
                    )
                elif df_formatted[col].dtype == 'object':
                    try:
                        numeric_series = pd.to_numeric(df_formatted[col], errors='coerce')
                        if not numeric_series.isna().all():
                            df_formatted[col] = numeric_series.apply(
                                lambda x: f"{x:.1f}" if pd.notna(x) and not pd.isna(x) else ""
                            )
                    except:
                        pass
            # Handle 2 decimal place columns (no % sign)
            elif is_two_decimal:
                if df_formatted[col].dtype in [np.int64, np.int32, np.float64, np.float32]:
                    df_formatted[col] = df_formatted[col].apply(
                        lambda x: f"{x:.2f}" if pd.notna(x) and not pd.isna(x) else ""
                    )
                elif df_formatted[col].dtype == 'object':
                    try:
                        numeric_series = pd.to_numeric(df_formatted[col], errors='coerce')
                        if not numeric_series.isna().all():
                            df_formatted[col] = numeric_series.apply(
                                lambda x: f"{x:.2f}" if pd.notna(x) and not pd.isna(x) else ""
                            )
                    except:
                        pass
            # Handle all other numeric columns: round to whole numbers with thousand separators
            else:
                if df_formatted[col].dtype in [np.int64, np.int32, np.float64, np.float32]:
                    df_formatted[col] = df_formatted[col].apply(
                        lambda x: f"{int(round(x)):,}" if pd.notna(x) and not pd.isna(x) else ""
                    )
                elif df_formatted[col].dtype == 'object':
                    try:
                        numeric_series = pd.to_numeric(df_formatted[col], errors='coerce')
                        if not numeric_series.isna().all():
                            df_formatted[col] = numeric_series.apply(
                                lambda x: f"{int(round(x)):,}" if pd.notna(x) and not pd.isna(x) else ""
                            )
                    except:
                        pass
        return df_formatted
    
    def apply_filters(df):
        """Apply active filters to DataFrame"""
        import re
        filtered_df = df.copy()
        
        # Apply non-numeric column filters
        for col, filter_info in filter_state.items():
            if col not in filtered_df.columns:
                continue
            if not filter_info['values']:  # No filters selected
                continue
            
            mode = filter_info['mode']
            values = filter_info['values']
            
            # Escape special regex characters
            escaped_values = [re.escape(str(v)) for v in values]
            pattern = '|'.join(escaped_values)
            
            if mode == 'contains':
                # Keep rows where column contains any of the selected values
                mask = filtered_df[col].astype(str).str.contains(pattern, case=False, na=False, regex=True)
                filtered_df = filtered_df[mask]
            elif mode == 'not_contains':
                # Keep rows where column does NOT contain any of the selected values
                mask = ~filtered_df[col].astype(str).str.contains(pattern, case=False, na=False, regex=True)
                filtered_df = filtered_df[mask]
        
        # Apply numeric column range filters
        for col, range_info in range_state.items():
            if col not in filtered_df.columns:
                continue
            
            current_min = range_info['current_min']
            current_max = range_info['current_max']
            col_min = range_info['min']
            col_max = range_info['max']
            
            # Only apply filter if range has been changed from default
            if current_min != col_min or current_max != col_max:
                try:
                    numeric_series = pd.to_numeric(filtered_df[col], errors='coerce')
                    mask = (numeric_series >= current_min) & (numeric_series <= current_max)
                    filtered_df = filtered_df[mask]
                except:
                    pass
        
        return filtered_df
    
    def update_display():
        """Update the displayed table - ensures only one table is displayed"""
        with output:
            # Clear output FIRST, inside the context manager
            clear_output(wait=True)
            
            # Apply filters first
            filtered_df = apply_filters(current_df)
            
            # Apply column visibility and order
            # Get visible columns in the specified order (Security always first)
            visible_cols = []
            if 'Security' in column_order and column_visibility.get('Security', True):
                visible_cols.append('Security')
            for col in column_order:
                if col != 'Security' and column_visibility.get(col, True):
                    visible_cols.append(col)
            
            # Select only visible columns in the specified order
            if visible_cols:
                display_df = filtered_df[visible_cols].copy()
            else:
                # If no columns visible, show all (shouldn't happen, but safety check)
                display_df = filtered_df.copy()
            
            # Always display 30 rows and reset index to remove it
            display_df = display_df.head(30).reset_index(drop=True)
            
            # Format numeric columns
            display_df = format_numeric_columns(display_df)
            
            # Build styled table with sticky Security column
            styled_table = display_df.style.set_table_styles([
                {'selector': 'thead th', 'props': [('background-color', '#2d2d2d'), 
                                                   ('color', '#ffffff'),
                                                   ('border', '1px solid #555555'),
                                                   ('white-space', 'nowrap'),
                                                   ('padding', '8px')]},
                {'selector': 'tbody td', 'props': [('background-color', '#1e1e1e'),
                                                   ('color', '#d4d4d4'),
                                                   ('border', '1px solid #555555'),
                                                   ('white-space', 'nowrap'),
                                                   ('padding', '8px')]},
                {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#252525')]},
                {'selector': 'tbody tr:hover', 'props': [('background-color', '#3c3c3c')]},
            ]).set_properties(**{
                'background-color': '#1e1e1e',
                'color': '#d4d4d4'
            })
            
            def add_sticky_security_column(html_str: str) -> str:
                """Add sticky class to the first column so it stays pinned."""
                soup = BeautifulSoup(html_str, 'html.parser')
                header = soup.find('thead')
                if header:
                    first_th = header.find('th')
                    if first_th:
                        classes = first_th.get('class', []) or []
                        if 'sticky-security' not in classes:
                            classes.append('sticky-security')
                        first_th['class'] = classes
                for tbody in soup.find_all('tbody'):
                    for row in tbody.find_all('tr'):
                        first_td = row.find('td')
                        if first_td:
                            classes = first_td.get('class', []) or []
                            if 'sticky-security' not in classes:
                                classes.append('sticky-security')
                            first_td['class'] = classes
                return str(soup)
            
            table_html = styled_table.to_html(classes='single-table', index=False)
            table_html = add_sticky_security_column(table_html)
            table_container = f"""
            <div class="single-table-container">
                {table_html}
            </div>
            """
            # Display ONLY ONCE inside the output widget
            display(HTML(table_container))
    
    def sort_column(col, ascending):
        """Sort by column"""
        sort_state['column'] = col
        sort_state['ascending'] = ascending
        nonlocal current_df
        current_df = df_original.sort_values(by=col, ascending=ascending, na_position='last')
        update_display()
    
    def clear_sorts(b):
        """Reset to original sort"""
        sort_state['column'] = None
        sort_state['ascending'] = True
        nonlocal current_df
        current_df = df_original.copy()
        update_display()
    
    def toggle_filter_mode(col):
        """Toggle between contains and does not contain mode"""
        if filter_state[col]['mode'] == 'contains':
            filter_state[col]['mode'] = 'not_contains'
        else:
            filter_state[col]['mode'] = 'contains'
        # Update button descriptions will happen on next display refresh
        update_display()
    
    def toggle_filter_value(col, value):
        """Toggle a filter value on/off"""
        if value in filter_state[col]['values']:
            filter_state[col]['values'].remove(value)
        else:
            filter_state[col]['values'].append(value)
        update_display()
    
    def clear_filters(b):
        """Clear all filters"""
        for col in filter_state:
            filter_state[col]['values'] = []
            filter_state[col]['mode'] = 'contains'
        update_display()
    
    def update_range(col, change):
        """Update range filter for a numeric column"""
        if col in range_state and 'new' in change:
            new_value = change['new']
            if isinstance(new_value, (tuple, list)) and len(new_value) == 2:
                range_state[col]['current_min'] = float(new_value[0])
                range_state[col]['current_max'] = float(new_value[1])
                update_display()
    
    def format_range_display(min_val, max_val, col_name):
        """Format range display string - show full range"""
        min_formatted = format_single_value(min_val, col_name)
        max_formatted = format_single_value(max_val, col_name)
        return f"{min_formatted} - {max_formatted}"
    
    def clear_ranges(b):
        """Reset all range sliders to default"""
        for col in range_state:
            range_state[col]['current_min'] = range_state[col]['min']
            range_state[col]['current_max'] = range_state[col]['max']
            # Update slider if it exists
            if col in slider_refs and slider_refs[col] is not None:
                slider = slider_refs[col]
                # Check if it's IntRangeSlider or FloatRangeSlider
                if isinstance(slider, widgets.IntRangeSlider):
                    slider.value = (int(range_state[col]['min']), int(range_state[col]['max']))
                else:
                    slider.value = (range_state[col]['min'], range_state[col]['max'])
            # Update readout if it exists (will auto-update via observer, but ensure it's correct)
            if col in readout_refs and readout_refs[col] is not None:
                readout_refs[col].value = format_range_display(
                    range_state[col]['min'], 
                    range_state[col]['max'], 
                    col
                )
        update_display()
    
    # Store slider and readout references for updating
    slider_refs = {}
    readout_refs = {}
    
    # Create buttons for each column with dark mode styling
    column_buttons = []
    
    for col in df.columns:
        # Create sort buttons
        asc_button = widgets.Button(
            description='↑',
            tooltip=f'Sort {col} ascending',
            button_style='',
            layout=widgets.Layout(width='30px', height='25px', 
                                 background_color='#3c3c3c',
                                 border='1px solid #555555')
        )
        
        desc_button = widgets.Button(
            description='↓',
            tooltip=f'Sort {col} descending',
            button_style='',
            layout=widgets.Layout(width='30px', height='25px',
                                 background_color='#3c3c3c',
                                 border='1px solid #555555')
        )
        
        # Bind sort click handlers
        asc_button.on_click(lambda b, c=col: sort_column(c, True))
        desc_button.on_click(lambda b, c=col: sort_column(c, False))
        
        # Create filter buttons for non-numeric columns
        filter_buttons = []
        if col in non_numeric_cols:
            # Get unique values for this column (limit to first 50 for performance)
            unique_vals = sorted([str(v) for v in df[col].dropna().unique() if pd.notna(v)])[:50]
            
            # Contains/Does not contain mode button
            mode_button = widgets.Button(
                description='⊃' if filter_state[col]['mode'] == 'contains' else '⊅',
                tooltip=f"Filter mode: {'Contains' if filter_state[col]['mode'] == 'contains' else 'Does not contain'}",
                button_style='info' if filter_state[col]['mode'] == 'contains' else 'warning',
                layout=widgets.Layout(width='30px', height='25px',
                                     background_color='#3c3c3c',
                                     border='1px solid #555555')
            )
            mode_button.on_click(lambda b, c=col: toggle_filter_mode(c))
            filter_buttons.append(mode_button)
            
            # Filter button to open filter selection
            filter_btn = widgets.Button(
                description='F',
                tooltip=f'Filter {col} - Click to select values',
                button_style='',
                layout=widgets.Layout(width='30px', height='25px',
                                     background_color='#3c3c3c',
                                     border='1px solid #555555')
            )
            
            # Create filter selection dropdown
            filter_dropdown = widgets.SelectMultiple(
                options=unique_vals,
                value=[str(v) for v in filter_state[col]['values']],
                description='',
                layout=widgets.Layout(width='200px', height='150px', display='none'),
                style={'background_color': '#1e1e1e', 'text_color': '#d4d4d4'}
            )
            
            def on_filter_change(change, col_name=col):
                filter_state[col_name]['values'] = list(change['new'])
                update_display()
            
            filter_dropdown.observe(lambda change, c=col: on_filter_change(change, c), names='value')
            
            # Store visibility state
            filter_dropdown._visible = False
            
            # Toggle filter dropdown visibility
            def toggle_filter_dropdown(b, col_name=col, dropdown=filter_dropdown):
                if dropdown._visible:
                    dropdown.layout.display = 'none'
                    dropdown._visible = False
                else:
                    dropdown.layout.display = 'block'
                    dropdown._visible = True
            
            filter_btn.on_click(lambda b, c=col, d=filter_dropdown: toggle_filter_dropdown(b, c, d))
            filter_buttons.append(filter_btn)
            filter_buttons.append(filter_dropdown)
        
        # Create range sliders for numeric columns
        range_buttons = []
        if col in range_state:
            range_info = range_state[col]
            col_min = range_info['min']
            col_max = range_info['max']
            current_min = range_info['current_min']
            current_max = range_info['current_max']
            
            # Determine if we should use IntRangeSlider or FloatRangeSlider
            # Use IntRangeSlider if all values are integers
            use_int = False
            try:
                numeric_series = pd.to_numeric(df[col], errors='coerce').dropna()
                if len(numeric_series) > 0:
                    # Check if values are close to integers
                    if all(abs(v - round(v)) < 0.0001 for v in numeric_series.head(100)):
                        use_int = True
            except:
                pass
            
            # Create formatted readout label (showing full formatted range)
            range_readout = widgets.Label(
                value=format_range_display(current_min, current_max, col),
                layout=widgets.Layout(width='300px', height='25px', margin='0px 5px', min_width='300px'),
                style={'text_color': '#ffffff', 'font_size': '12px'}
            )
            
            # Store readout reference
            readout_refs[col] = range_readout
            
            if use_int:
                range_slider = widgets.IntRangeSlider(
                    value=(int(current_min), int(current_max)),
                    min=int(col_min),
                    max=int(col_max),
                    step=1,
                    description='',
                    readout=False,  # Hide default readout
                    layout=widgets.Layout(width='250px', height='25px'),
                    style={'handle_color': '#6495ed'}
                )
            else:
                # Calculate step size (1% of range, with minimum step)
                step_size = max((col_max - col_min) / 100, (col_max - col_min) / 1000)
                range_slider = widgets.FloatRangeSlider(
                    value=(current_min, current_max),
                    min=col_min,
                    max=col_max,
                    step=step_size,
                    description='',
                    readout=False,  # Hide default readout
                    layout=widgets.Layout(width='250px', height='25px'),
                    style={'handle_color': '#6495ed'}
                )
            
            # Update readout when slider changes
            def update_readout(change, col_name=col, readout=range_readout):
                if 'new' in change:
                    new_value = change['new']
                    if isinstance(new_value, (tuple, list)) and len(new_value) == 2:
                        readout.value = format_range_display(new_value[0], new_value[1], col_name)
            
            # Observe slider changes
            range_slider.observe(lambda change, c=col: update_range(c, change), names='value')
            range_slider.observe(lambda change, c=col, r=range_readout: update_readout(change, c, r), names='value')
            
            # Store slider reference for clearing
            slider_refs[col] = range_slider
            
            # Create container with slider and readout
            slider_container = widgets.HBox([
                range_slider,
                range_readout
            ], layout=widgets.Layout(background_color='#1e1e1e'))
            
            range_buttons.append(slider_container)
        
        # Create column header with buttons
        col_elements = [
            widgets.Label(col, layout=widgets.Layout(width='150px', color='#d4d4d4')),
            asc_button,
            desc_button
        ]
        col_elements.extend(filter_buttons)
        col_elements.extend(range_buttons)
        
        col_header = widgets.HBox(
            col_elements,
            layout=widgets.Layout(background_color='#1e1e1e')
        )
        
        column_buttons.append(col_header)
    
    # Create clear all sorts button with dark styling
    clear_sorts_button = widgets.Button(
        description='Clear All Sorts',
        button_style='warning',
        layout=widgets.Layout(width='200px', margin='10px 0px',
                             background_color='#856404',
                             border='1px solid #555555')
    )
    clear_sorts_button.on_click(clear_sorts)
    
    # Create clear all filters button with dark styling
    clear_filters_button = widgets.Button(
        description='Clear All Filters',
        button_style='warning',
        layout=widgets.Layout(width='200px', margin='10px 0px',
                             background_color='#856404',
                             border='1px solid #555555')
    )
    clear_filters_button.on_click(clear_filters)
    
    # Create clear all ranges button with dark styling
    clear_ranges_button = widgets.Button(
        description='Clear All Ranges',
        button_style='warning',
        layout=widgets.Layout(width='200px', margin='10px 0px',
                             background_color='#856404',
                             border='1px solid #555555')
    )
    clear_ranges_button.on_click(clear_ranges)
    
    # Column selector panel - using HTML/JS for drag-and-drop with proper Python integration
    column_selector_visible = False
    column_selector_output = widgets.Output()
    
    # Store column widget references for updates
    column_widget_refs = {}
    
    def move_column_up(col):
        """Move column up in order"""
        nonlocal column_order
        if col == 'Security' or col not in column_order:
            return
        idx = column_order.index(col)
        if idx > 1:  # Can't move before Security (index 0)
            column_order[idx], column_order[idx-1] = column_order[idx-1], column_order[idx]
            refresh_column_selector()
            update_display()
    
    def move_column_down(col):
        """Move column down in order"""
        nonlocal column_order
        if col == 'Security' or col not in column_order:
            return
        idx = column_order.index(col)
        if idx < len(column_order) - 1:
            column_order[idx], column_order[idx+1] = column_order[idx+1], column_order[idx]
            refresh_column_selector()
            update_display()
    
    def toggle_column_visibility(col, change):
        """Toggle column visibility"""
        if col != 'Security':  # Security is always visible
            new_value = change['new']
            column_visibility[col] = new_value
            update_display()
    
    def refresh_column_selector():
        """Refresh the column selector panel"""
        if column_selector_visible:
            with column_selector_output:
                clear_output(wait=True)
                create_column_selector_widgets()
    
    
    
    def create_column_selector_widgets():
        """Create column selector using pure ipywidgets - more reliable"""
        column_items = []
        
        for col in column_order:
            is_visible = column_visibility.get(col, True)
            is_security = (col == 'Security')
            
            # Checkbox for visibility
            checkbox = widgets.Checkbox(
                value=is_visible,
                description='',
                disabled=is_security,
                layout=widgets.Layout(width='30px'),
                style={'description_width': '0px'}
            )
            if not is_security:
                checkbox.observe(lambda change, c=col: toggle_column_visibility(c, change), names='value')
            
            # Column name label - try multiple approaches to ensure visibility
            # First try: Label with fixed width
            try:
                name_label = widgets.Label(
                    value=col,
                    layout=widgets.Layout(
                        width='180px',  # Fixed width instead of auto
                        margin='0px 5px',
                        overflow='hidden',
                        visibility='visible',
                        display='flex'
                    )
                )
                name_label.style.text_color = '#d4d4d4'
                name_label.style.font_size = '13px'
            except:
                # Fallback: HTML widget with inline styles
                name_label = widgets.HTML(
                    value=f'<div style="color: #d4d4d4 !important; font-size: 13px !important; width: 180px; display: inline-block; white-space: nowrap; overflow: hidden; text-overflow: ellipsis;">{col}</div>',
                    layout=widgets.Layout(
                        width='180px',
                        margin='0px 5px'
                    )
                )
            
            # Drag handle icon
            drag_handle = widgets.HTML(
                value='<span style="color: #888; font-size: 16px; cursor: move;">☰</span>',
                layout=widgets.Layout(width='25px', margin='0px 5px')
            )
            
            # Up/Down buttons for reordering (not for Security)
            if is_security:
                item = widgets.HBox([
                    drag_handle,
                    checkbox,
                    name_label
                ], layout=widgets.Layout(
                    background_color='#2d2d2d', 
                    border='1px solid #555555', 
                    padding='8px', 
                    margin='3px 0px',
                    width='300px',
                    max_width='300px',
                    align_items='center'
                ))
            else:
                up_btn = widgets.Button(
                    description='↑',
                    tooltip=f'Move {col} up',
                    layout=widgets.Layout(width='35px', height='28px', margin='0px 2px'),
                    button_style='',
                    style={'button_color': '#3c3c3c'}
                )
                up_btn.on_click(lambda b, c=col: move_column_up(c))
                
                down_btn = widgets.Button(
                    description='↓',
                    tooltip=f'Move {col} down',
                    layout=widgets.Layout(width='35px', height='28px', margin='0px 2px'),
                    button_style='',
                    style={'button_color': '#3c3c3c'}
                )
                down_btn.on_click(lambda b, c=col: move_column_down(c))
                
                item = widgets.HBox([
                    drag_handle,
                    checkbox,
                    name_label,
                    up_btn,
                    down_btn
                ], layout=widgets.Layout(
                    background_color='#2d2d2d', 
                    border='1px solid #555555', 
                    padding='8px', 
                    margin='3px 0px',
                    width='300px',
                    max_width='300px',
                    align_items='center'
                ))
            
            column_items.append(item)
        
        # Reset button
        reset_btn = widgets.Button(
            description='Reset to Default',
            button_style='warning',
            layout=widgets.Layout(width='200px', margin='10px 0px'),
            style={'button_color': '#856404'}
        )
        reset_btn.on_click(lambda b: reset_columns_to_default())
        
        # Panel header - make sure it's visible
        header_title = widgets.HTML(
            value="<h4 style='color: #ffffff; margin: 0; font-weight: bold;'>Column Selector</h4>",
            layout=widgets.Layout(margin='0px 5px')
        )
        header = widgets.HBox([
            header_title,
            reset_btn
        ], layout=widgets.Layout(
            justify_content='space-between', 
            width='300px',
            max_width='300px',
            align_items='center',
            margin='0px 0px 10px 0px'
        ))
        
        # Ensure we have items to display
        if not column_items:
            return
        
        # Panel container - fixed width so it doesn't expand
        panel = widgets.VBox([
            header,
            widgets.HTML("<hr style='border-color: #555555; margin: 10px 0;'>"),
            *column_items
        ], layout=widgets.Layout(
            background_color='#1e1e1e',
            border='1px solid #555555',
            padding='10px',
            width='320px',
            min_width='320px',
            max_width='320px',
            max_height='600px',
            overflow_y='auto',
            margin='0px 0px 0px 10px'
        ))
        
        display(panel)
    
    def toggle_column_selector(b):
        """Toggle column selector panel visibility"""
        nonlocal column_selector_visible
        column_selector_visible = not column_selector_visible
        # Adjust container width based on visibility
        if column_selector_visible:
            column_selector_container.layout.width = '340px'
        else:
            column_selector_container.layout.width = '0px'
        with column_selector_output:
            clear_output(wait=True)
            if column_selector_visible:
                create_column_selector_widgets()
    
    def reset_columns_to_default(b=None):
        """Reset columns to default order and visibility"""
        nonlocal column_order, column_visibility
        column_order = default_column_order.copy()
        column_visibility = {col: True for col in df.columns}
        refresh_column_selector()
        update_display()
    
    # Create toggle button for column selector
    column_selector_toggle = widgets.Button(
        description='⚙ Columns',
        tooltip='Toggle column selector panel',
        button_style='',
        layout=widgets.Layout(width='150px', margin='10px 0px',
                             background_color='#3c3c3c',
                             border='1px solid #555555')
    )
    column_selector_toggle.on_click(toggle_column_selector)
    
    # Create layout with dark mode container
    header_info = widgets.HBox([
        widgets.HTML("<h3 style='color: #ffffff;'>Click ↑ or ↓ to sort | Click F to filter non-numeric columns | Use sliders for numeric columns</h3>"),
        clear_sorts_button,
        clear_filters_button,
        clear_ranges_button,
        column_selector_toggle
    ], layout=widgets.Layout(background_color='#1e1e1e'))
    
    # Group column buttons in a scrollable area
    buttons_container = widgets.VBox([
        header_info,
        widgets.HTML("<hr style='border-color: #555555;'>"),
        *column_buttons
    ], layout=widgets.Layout(
        background_color='#1e1e1e', 
        flex='1 1 auto',
        min_width='400px',
        width='auto',
        padding='0px',
        margin='0px'
    ))
    
    # Column selector output container - only takes space when visible
    column_selector_container = widgets.VBox([
        column_selector_output
    ], layout=widgets.Layout(
        width='0px',
        min_width='0px',
        max_width='340px',
        flex='0 0 auto',
        padding='0px',
        margin='0px',
        overflow='hidden'
    ))
    
    # Main controls area with column selector on the right
    main_controls = widgets.HBox([
        buttons_container,
        column_selector_container
    ], layout=widgets.Layout(
        background_color='#1e1e1e',
        width='100%',
        justify_content='flex-start',
        align_items='flex-start',
        padding='0px',
        margin='0px'
    ))
    
    # Initial display
    update_display()
    
    # Display everything
    display(widgets.VBox([
        main_controls,
        widgets.HTML("<hr style='border-color: #555555;'>"),
        output
    ], layout=widgets.Layout(
        background_color='#1e1e1e',
        width='100%',
        padding='0px',
        margin='0px'
    )))

# Create and display the sortable table
create_sortable_table(df_clean)